In [1]:
# ============================================================================
# CELL 1: DATA LOADING AND PREPARATION
# ============================================================================
print("="*80)
print("📁 DATA LOADING AND PREPARATION")
print("="*80)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

# ============================================================================
# LOAD YOUR ACTUAL DATA
# ============================================================================
print("📂 Loading your actual insurance data...")

# Use the exact path from your directory listing
data_path = Path("C:/Users/G5/Desktop/Insurance Anlytics/data/01_interim/cleaned_data.txt")
print(f"   • Data path: {data_path}")

# First, let's check the file size and preview the first few lines
print(f"   • File size: {data_path.stat().st_size / (1024*1024):.2f} MB")

# Read the first few lines to understand the format
with open(data_path, 'r', encoding='utf-8') as f:
    first_lines = [f.readline() for _ in range(5)]

print("\n🔍 Preview of first few lines in the file:")
for i, line in enumerate(first_lines, 1):
    print(f"   Line {i}: {line[:100]}...")  # Show first 100 chars

# Try different delimiters
delimiters_to_try = [',', '\t', ';', '|']

for delimiter in delimiters_to_try:
    try:
        print(f"\n🔍 Trying delimiter: '{repr(delimiter)}'")
        
        # Read a sample first to check structure
        sample_df = pd.read_csv(data_path, nrows=100, sep=delimiter, encoding='utf-8')
        
        print(f"   • Success! Found {sample_df.shape[1]} columns")
        print(f"   • Column names: {list(sample_df.columns)}")
        
        # Load the full data with the correct delimiter
        analysis_df = pd.read_csv(data_path, sep=delimiter, encoding='utf-8')
        
        print(f"\n✅ SUCCESS! Data loaded with delimiter: '{repr(delimiter)}'")
        print(f"   • Total records: {analysis_df.shape[0]:,}")
        print(f"   • Total columns: {analysis_df.shape[1]}")
        break
        
    except Exception as e:
        print(f"   • Failed with delimiter '{repr(delimiter)}': {str(e)[:100]}")

# If none of the standard delimiters worked, try to detect it
if 'analysis_df' not in locals() or analysis_df is None:
    print("\n🔍 Standard delimiters didn't work. Trying to auto-detect...")
    
    # Read the entire file as text
    with open(data_path, 'r', encoding='utf-8') as f:
        content = f.read(10000)  # Read first 10KB
    
    # Try to find the most common separator
    import re
    
    # Common separators to check
    separators = [',', '\t', ';', '|', ':', ' ', '\s+']
    
    for sep in separators:
        pattern = sep if sep != '\s+' else r'\s+'
        if len(re.split(pattern, content.split('\n')[0])) > 1:
            print(f"   • Found potential separator: {repr(sep)}")
            try:
                analysis_df = pd.read_csv(data_path, sep=pattern, engine='python', encoding='utf-8')
                print(f"✅ Successfully loaded with separator pattern: {repr(sep)}")
                break
            except:
                continue

# If still no success, try reading with pandas' auto-detection
if 'analysis_df' not in locals() or analysis_df is None:
    print("\n🔍 Trying pandas auto-detection...")
    try:
        analysis_df = pd.read_csv(data_path, sep=None, engine='python', encoding='utf-8')
        print(f"✅ Loaded using pandas auto-detection")
    except Exception as e:
        print(f"❌ Failed to load data: {e}")
        print("\n📋 Alternative loading options:")
        print("   1. Check if file is in a different format (JSON, Excel)")
        print("   2. Check file encoding")
        print("   3. Inspect the raw file structure")
        
        # Show more of the file structure
        print("\n🔍 File structure analysis:")
        with open(data_path, 'r', encoding='utf-8', errors='ignore') as f:
            lines = [next(f) for _ in range(10)]
        
        for i, line in enumerate(lines, 1):
            print(f"Line {i} (length: {len(line)}): {repr(line[:100])}")

# ============================================================================
# DATA CLEANUP AND PREPARATION
# ============================================================================
print("\n" + "="*80)
print("🧹 DATA CLEANUP AND PREPARATION")
print("="*80)

if 'analysis_df' in locals() and analysis_df is not None:
    # Clean column names (remove spaces, special characters)
    analysis_df.columns = [str(col).strip().replace(' ', '_').replace('-', '_').replace('.', '_').lower() 
                          for col in analysis_df.columns]
    
    print("📋 Data Overview:")
    print(f"   • Shape: {analysis_df.shape[0]:,} rows × {analysis_df.shape[1]} columns")
    print(f"   • Memory usage: {analysis_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
    
    # Show basic info
    print("\n📊 Data Types:")
    print(analysis_df.dtypes.value_counts())
    
    print("\n🔍 First 5 rows:")
    display(analysis_df.head())
    
    print("\n🔍 Last 5 rows:")
    display(analysis_df.tail())
    
    print("\n📈 Basic Statistics:")
    numeric_cols = analysis_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(analysis_df[numeric_cols].describe().round(2))
    else:
        print("   No numeric columns found")
    
    print("\n🔍 Missing Values:")
    missing = analysis_df.isnull().sum()
    missing_pct = (missing / len(analysis_df) * 100).round(2)
    missing_df = pd.DataFrame({'Missing_Count': missing, 'Missing_%': missing_pct})
    display(missing_df[missing_df['Missing_Count'] > 0])
    
    if missing.sum() == 0:
        print("   ✅ No missing values found!")
    
    # Check for duplicates
    duplicates = analysis_df.duplicated().sum()
    print(f"\n🔍 Duplicates: {duplicates:,} ({duplicates/len(analysis_df)*100:.2f}%)")
    
    if duplicates > 0:
        print("   ⚠️ Duplicates found - consider removing them")
    else:
        print("   ✅ No duplicates found")
    
    # Show unique values for categorical columns
    categorical_cols = analysis_df.select_dtypes(include=['object', 'category']).columns
    print(f"\n🔍 Categorical Columns ({len(categorical_cols)}):")
    for col in categorical_cols[:5]:  # Show first 5
        unique_vals = analysis_df[col].nunique()
        print(f"   • {col}: {unique_vals} unique values")
        if unique_vals <= 10:
            print(f"     Values: {analysis_df[col].unique().tolist()}")
    
    # Custom color palettes
    print("\n🎨 Setting up visualization palettes...")
    risk_palette = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6B8F71']
    sequential_palette = px.colors.sequential.Viridis
    diverging_palette = px.colors.diverging.RdYlGn_r
    
    print(f"\n✅ Data preparation complete!")
    print(f"   • Data shape: {analysis_df.shape}")
    print(f"   • Ready for outlier detection analysis")
else:
    print("\n❌ Failed to load data. Please check your file format and try again.")
    print("   You might need to:")
    print("   1. Convert your .txt file to .csv format")
    print("   2. Check the delimiter used in the file")
    print("   3. Clean the file structure manually")

print("\n" + "="*80)
print("📁 DATA LOADING COMPLETE")
print("="*80)

📁 DATA LOADING AND PREPARATION
📂 Loading your actual insurance data...
   • Data path: C:\Users\G5\Desktop\Insurance Anlytics\data\01_interim\cleaned_data.txt
   • File size: 538.51 MB

🔍 Preview of first few lines in the file:
   Line 1: underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|b...
   Line 2: 145249|12827|2015-03-01|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not...
   Line 3: 145249|12827|2015-05-01|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not...
   Line 4: 145249|12827|2015-07-01|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not...
   Line 5: 145255|12827|2015-05-01|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not...

🔍 Trying delimiter: '',''
   • Success! Found 1 columns
   • Column names: ['underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|mar

,underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims
0,145249|12827|2015-03-01|True| |Close Corporat...
1,145249|12827|2015-05-01|True| |Close Corporat...
2,145249|12827|2015-07-01|True| |Close Corporat...
3,145255|12827|2015-05-01|True| |Close Corporat...
4,145255|12827|2015-07-01|True| |Close Corporat...



🔍 Last 5 rows:


,underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims
1000093,31520|389|2015-04-01|False|ZW|Individual|Mr|En...
1000094,31520|389|2015-06-01|False|ZW|Individual|Mr|En...
1000095,31520|389|2015-08-01|False|ZW|Individual|Mr|En...
1000096,31519|389|2014-07-01|False|ZW|Individual|Mr|En...
1000097,31519|389|2015-02-01|False|ZW|Individual|Mr|En...



📈 Basic Statistics:
   No numeric columns found

🔍 Missing Values:


,Missing_Count,Missing_%


   ✅ No missing values found!

🔍 Duplicates: 74 (0.01%)
   ⚠️ Duplicates found - consider removing them

🔍 Categorical Columns (1):
   • underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims: 1000024 unique values

🎨 Setting up visualization palettes...

✅ Data preparation complete!
   • Data shape: (1000098, 1)
   • Ready for outlier detection analysis

📁 DATA LOADING COMPLETE


In [2]:
# ============================================================================
# CELL 2: CHECK FOR REQUIRED COLUMNS
# ============================================================================
print("="*80)
print("🔍 CHECKING FOR REQUIRED COLUMNS")
print("="*80)

# Show all columns
print("📋 All available columns:")
for i, col in enumerate(analysis_df.columns, 1):
    print(f"   {i:2d}. {col}")

# Look for specific column patterns
print("\n🔍 Searching for key columns...")

# Look for Province/State columns
province_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                     for word in ['province', 'state', 'region', 'location', 'geo', 'area'])]
print(f"   • Province candidates: {province_candidates}")

# Look for VehicleType columns
vehicle_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                    for word in ['vehicle', 'veh', 'car', 'auto', 'type', 'make', 'model'])]
print(f"   • Vehicle candidates: {vehicle_candidates}")

# Look for Premium columns
premium_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                     for word in ['premium', 'price', 'cost', 'amount', 'value'])]
print(f"   • Premium candidates: {premium_candidates}")

# Look for Claims columns
claims_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                    for word in ['claim', 'loss', 'payout', 'damage', 'cost'])]
print(f"   • Claims candidates: {claims_candidates}")

# Check for other important columns
print("\n🔍 Checking for other important columns:")

# Numeric columns
numeric_cols = analysis_df.select_dtypes(include=[np.number]).columns.tolist()
print(f"   • Numeric columns ({len(numeric_cols)}): {numeric_cols}")

# Categorical columns
categorical_cols = analysis_df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"   • Categorical columns ({len(categorical_cols)}): {categorical_cols}")

# Date columns
date_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                   for word in ['date', 'time', 'year', 'month', 'day'])]
print(f"   • Date candidates: {date_candidates}")

# ID columns
id_candidates = [col for col in analysis_df.columns if any(word in col.lower() 
                 for word in ['id', 'code', 'num', 'number', 'ref', 'policy'])]
print(f"   • ID candidates: {id_candidates}")

# Check data types
print("\n📊 Data Types Summary:")
dtype_summary = analysis_df.dtypes.value_counts()
for dtype, count in dtype_summary.items():
    print(f"   • {dtype}: {count} columns")

# Check for missing values
print("\n🔍 Missing Values Summary:")
missing_summary = analysis_df.isnull().sum()
total_missing = missing_summary.sum()
print(f"   • Total missing values: {total_missing:,}")
print(f"   • Percentage of total data: {(total_missing / (analysis_df.shape[0] * analysis_df.shape[1]) * 100):.2f}%")

# Show columns with highest missing values (top 10)
if total_missing > 0:
    missing_df = pd.DataFrame({
        'column': missing_summary.index,
        'missing_count': missing_summary.values,
        'missing_pct': (missing_summary.values / len(analysis_df) * 100)
    })
    missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)
    
    print(f"\n📊 Top columns with missing values:")
    display(missing_df.head(10))
else:
    print("   ✅ No missing values found!")

# Check for unique values in categorical columns
print("\n🔍 Categorical Columns Analysis:")
if categorical_cols:
    for col in categorical_cols[:10]:  # Show first 10 categorical columns
        unique_vals = analysis_df[col].nunique()
        print(f"   • {col}: {unique_vals} unique values")
        if unique_vals <= 15:  # Show values if not too many
            print(f"     Values: {analysis_df[col].unique().tolist()}")
else:
    print("   • No categorical columns found")

# Check basic statistics for numeric columns
print("\n📈 Numeric Columns Basic Statistics:")
if numeric_cols:
    # Select first 5 numeric columns for summary
    cols_to_show = numeric_cols[:min(5, len(numeric_cols))]
    stats_df = analysis_df[cols_to_show].describe().round(2)
    display(stats_df)
    
    # Check for zeros and negative values
    print(f"\n🔍 Zero/negative value check for numeric columns:")
    for col in cols_to_show:
        zero_count = (analysis_df[col] == 0).sum()
        negative_count = (analysis_df[col] < 0).sum()
        
        if zero_count > 0:
            zero_pct = (zero_count / len(analysis_df) * 100)
            print(f"   • {col}: {zero_count:,} zeros ({zero_pct:.1f}%)")
        
        if negative_count > 0:
            negative_pct = (negative_count / len(analysis_df) * 100)
            print(f"   • {col}: {negative_count:,} negative values ({negative_pct:.1f}%)")
else:
    print("   • No numeric columns found")

# Create visualization dataframe
viz_df = analysis_df.copy()
print(f"\n✅ Created visualization dataframe: {viz_df.shape}")

# Add recommendations based on findings
print("\n" + "="*80)
print("💡 RECOMMENDATIONS FOR OUTLIER ANALYSIS")
print("="*80)

recommendations = []

# Check if we have enough numeric columns for outlier analysis
if len(numeric_cols) >= 3:
    print("✅ Good for outlier analysis - sufficient numeric columns found")
    print(f"   • Available numeric columns: {len(numeric_cols)}")
    
    # Identify potential target columns for analysis
    potential_targets = []
    for col in numeric_cols:
        col_mean = analysis_df[col].mean()
        col_std = analysis_df[col].std()
        
        # Check if column has variability
        if col_std > 0 and col_mean != 0:
            cv = col_std / col_mean  # Coefficient of variation
            if cv > 0.1:  # At least 10% variability
                potential_targets.append(col)
    
    print(f"   • High-variability columns for outlier detection: {len(potential_targets)}")
    if potential_targets:
        print(f"   • Suggested columns: {potential_targets[:5]}")  # Top 5
    
    recommendations.append("Use 3-5 high-variability numeric columns for ML outlier detection")
else:
    print("⚠️ Limited numeric columns for outlier analysis")
    print(f"   • Only {len(numeric_cols)} numeric columns found")
    recommendations.append("Consider creating derived numeric features for better outlier detection")

# Check for categorical columns that could be useful
if categorical_cols:
    categorical_info = []
    for col in categorical_cols[:5]:  # First 5 categorical columns
        unique_count = analysis_df[col].nunique()
        if 2 <= unique_count <= 20:  # Good for grouping/filtering
            categorical_info.append(f"{col} ({unique_count} categories)")
    
    if categorical_info:
        print(f"✅ Useful categorical columns found:")
        for info in categorical_info:
            print(f"   • {info}")
        recommendations.append("Use categorical columns for segmentation in outlier analysis")

# Check data size
if len(analysis_df) > 100000:
    print(f"⚠️ Large dataset: {len(analysis_df):,} rows")
    recommendations.append("Consider sampling for faster outlier detection")
elif len(analysis_df) < 1000:
    print(f"⚠️ Small dataset: {len(analysis_df):,} rows")
    recommendations.append("Use conservative outlier thresholds for small dataset")

# Check for potential date columns
if date_candidates:
    print(f"✅ Date columns found: {date_candidates}")
    recommendations.append("Consider temporal outlier analysis if dates are available")

# Print recommendations
print(f"\n📋 Recommended next steps:")
for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print("\n" + "="*80)
print("✅ COLUMN ANALYSIS COMPLETE")
print("="*80)
print(f"\n📊 Summary:")
print(f"   • Total columns: {analysis_df.shape[1]}")
print(f"   • Numeric columns: {len(numeric_cols)}")
print(f"   • Categorical columns: {len(categorical_cols)}")
print(f"   • Potential target columns: {len(premium_candidates + claims_candidates)}")

# Store available metrics for later use
available_metrics = numeric_cols  # Use numeric columns as metrics for outlier detection
print(f"\n💾 Available metrics for outlier detection: {len(available_metrics)} variables")
print(f"   Metrics: {available_metrics}")

print(f"\n🚀 Ready for next step: Univariate Outlier Detection")

🔍 CHECKING FOR REQUIRED COLUMNS
📋 All available columns:
    1. underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims

🔍 Searching for key columns...
   • Province candidates: ['underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrat

In [3]:
# ============================================================================
# CELL 3: DATA ENHANCEMENT FOR VISUALIZATION
# ============================================================================
print("="*80)
print("✨ ENHANCING DATA FOR VISUALIZATION")
print("="*80)

# Create a copy to avoid modifying the original
viz_df = analysis_df.copy()

# First, let's check the data types of potential premium and claims columns
print("🔍 Checking data types for potential numeric columns...")

# Get all potential numeric columns
all_numeric_cols = viz_df.select_dtypes(include=[np.number]).columns.tolist()
print(f"✅ Verified numeric columns: {len(all_numeric_cols)}")
for i, col in enumerate(all_numeric_cols[:10], 1):  # Show first 10
    print(f"   {i:2d}. {col}")

# Find actual numeric premium columns (not just by name, but by type)
numeric_premium_cols = []
string_premium_cols = []

for col in viz_df.columns:
    if any(word in col.lower() for word in ['premium', 'price', 'cost', 'amount', 'value']):
        if pd.api.types.is_numeric_dtype(viz_df[col]):
            numeric_premium_cols.append(col)
        else:
            string_premium_cols.append(col)

print(f"\n🔍 Premium column analysis:")
print(f"   • Numeric premium columns: {numeric_premium_cols}")
print(f"   • String premium columns: {string_premium_cols}")

# Find actual numeric claims columns
numeric_claims_cols = []
string_claims_cols = []

for col in viz_df.columns:
    if any(word in col.lower() for word in ['claim', 'loss', 'payout', 'damage', 'payment']):
        if pd.api.types.is_numeric_dtype(viz_df[col]):
            numeric_claims_cols.append(col)
        else:
            string_claims_cols.append(col)

print(f"\n🔍 Claims column analysis:")
print(f"   • Numeric claims columns: {numeric_claims_cols}")
print(f"   • String claims columns: {string_claims_cols}")

# Try to convert string columns to numeric if possible
if string_premium_cols:
    print(f"\n🔄 Attempting to convert string premium columns to numeric...")
    for col in string_premium_cols[:3]:  # Try first 3
        try:
            # Try to convert to numeric
            converted = pd.to_numeric(viz_df[col], errors='coerce')
            if converted.notna().sum() > len(viz_df) * 0.5:  # If at least 50% convert successfully
                viz_df[col] = converted
                numeric_premium_cols.append(col)
                print(f"   ✅ Converted '{col}' to numeric")
            else:
                print(f"   ⚠️  '{col}' - insufficient numeric values after conversion")
        except Exception as e:
            print(f"   ❌ '{col}' - conversion failed: {e}")

if string_claims_cols:
    print(f"\n🔄 Attempting to convert string claims columns to numeric...")
    for col in string_claims_cols[:3]:  # Try first 3
        try:
            converted = pd.to_numeric(viz_df[col], errors='coerce')
            if converted.notna().sum() > len(viz_df) * 0.5:
                viz_df[col] = converted
                numeric_claims_cols.append(col)
                print(f"   ✅ Converted '{col}' to numeric")
            else:
                print(f"   ⚠️  '{col}' - insufficient numeric values after conversion")
        except Exception as e:
            print(f"   ❌ '{col}' - conversion failed: {e}")

# Now try to calculate Loss Ratio with verified numeric columns
if numeric_premium_cols and numeric_claims_cols:
    # Use the first available numeric premium and claims columns
    premium_col = numeric_premium_cols[0]
    claims_col = numeric_claims_cols[0]
    
    print(f"\n✅ Calculating Loss Ratio using:")
    print(f"   • Premium column: {premium_col}")
    print(f"   • Claims column: {claims_col}")
    
    # Display some statistics before calculation
    print(f"\n📊 Column Statistics:")
    print(f"   • {premium_col} - Mean: {viz_df[premium_col].mean():.2f}, Std: {viz_df[premium_col].std():.2f}")
    print(f"   • {claims_col} - Mean: {viz_df[claims_col].mean():.2f}, Std: {viz_df[claims_col].std():.2f}")
    
    # Calculate Loss Ratio with safety checks
    # Handle division by zero and ensure both columns are numeric
    try:
        # Convert to float to ensure numeric division
        premium_series = pd.to_numeric(viz_df[premium_col], errors='coerce')
        claims_series = pd.to_numeric(viz_df[claims_col], errors='coerce')
        
        # Avoid division by zero
        mask = (premium_series != 0) & (premium_series.notna()) & (claims_series.notna())
        
        viz_df['LossRatio'] = np.nan  # Initialize with NaN
        viz_df.loc[mask, 'LossRatio'] = claims_series[mask] / premium_series[mask]
        
        # Remove infinite values
        viz_df['LossRatio'] = viz_df['LossRatio'].replace([np.inf, -np.inf], np.nan)
        
        # Display statistics
        valid_loss_ratios = viz_df['LossRatio'].dropna()
        if len(valid_loss_ratios) > 0:
            print(f"\n✅ Successfully created LossRatio column")
            print(f"   • Valid LossRatio values: {len(valid_loss_ratios):,}")
            print(f"   • LossRatio range: {valid_loss_ratios.min():.4f} to {valid_loss_ratios.max():.4f}")
            print(f"   • LossRatio mean: {valid_loss_ratios.mean():.4f}")
            print(f"   • LossRatio median: {valid_loss_ratios.median():.4f}")
            print(f"   • NaN values: {viz_df['LossRatio'].isna().sum():,}")
        else:
            print(f"⚠️  No valid LossRatio values calculated")
            
    except Exception as e:
        print(f"❌ Error calculating LossRatio: {e}")
else:
    print(f"\n⚠️  Cannot calculate LossRatio:")
    if not numeric_premium_cols:
        print(f"   • No numeric premium columns found")
    if not numeric_claims_cols:
        print(f"   • No numeric claims columns found")
    
    # Show alternative: use any numeric columns
    print(f"\n🔍 Available numeric columns for alternative metrics:")
    if len(all_numeric_cols) >= 2:
        col1, col2 = all_numeric_cols[0], all_numeric_cols[1]
        print(f"   • Using {col1} and {col2} for ratio calculation")
        
        # Calculate ratio of first two numeric columns
        mask = (viz_df[col1] != 0) & (viz_df[col1].notna()) & (viz_df[col2].notna())
        viz_df['ValueRatio'] = np.nan
        viz_df.loc[mask, 'ValueRatio'] = viz_df.loc[mask, col2] / viz_df.loc[mask, col1]
        viz_df['ValueRatio'] = viz_df['ValueRatio'].replace([np.inf, -np.inf], np.nan)
        print(f"✅ Created ValueRatio column from {col2}/{col1}")

# Identify geographic column
geo_cols = [col for col in viz_df.columns if any(word in col.lower() for word in 
            ['province', 'state', 'region', 'city', 'location', 'area', 'geo'])]
if geo_cols:
    geo_col = geo_cols[0]
    viz_df['Geography'] = viz_df[geo_col].astype(str)
    print(f"\n✅ Using '{geo_col}' as Geography")
    
    # Show unique values (first 10)
    unique_geo = viz_df['Geography'].unique()[:10]
    print(f"   • Unique values (first 10): {list(unique_geo)}")

# Identify vehicle column
vehicle_cols = [col for col in viz_df.columns if any(word in col.lower() for word in 
               ['vehicle', 'car', 'auto', 'make', 'model', 'type', 'brand'])]
if vehicle_cols:
    vehicle_col = vehicle_cols[0]
    viz_df['VehicleCategory'] = viz_df[vehicle_col].astype(str)
    print(f"✅ Using '{vehicle_col}' as VehicleCategory")
    
    # Show unique values (first 10)
    unique_vehicle = viz_df['VehicleCategory'].unique()[:10]
    print(f"   • Unique values (first 10): {list(unique_vehicle)}")

# Create risk categories if we have a suitable ratio column
ratio_columns = ['LossRatio', 'ValueRatio'] if 'ValueRatio' in viz_df.columns else ['LossRatio']
ratio_column = None

for col in ratio_columns:
    if col in viz_df.columns:
        valid_values = viz_df[col].dropna()
        if len(valid_values) > 0:
            ratio_column = col
            break

if ratio_column:
    print(f"\n🎯 Creating RiskLevel categories using '{ratio_column}'")
    
    # Clean the ratio data
    ratio_data = viz_df[ratio_column].dropna()
    
    if len(ratio_data) > 0:
        print(f"   • Available values: {len(ratio_data):,}")
        print(f"   • Statistical summary:")
        print(ratio_data.describe())
        
        # Determine the best way to create categories
        unique_count = ratio_data.nunique()
        
        if unique_count >= 4:
            try:
                # Use quantile-based categories
                viz_df['RiskLevel'] = pd.qcut(
                    viz_df[ratio_column], 
                    q=4, 
                    labels=['Low Risk', 'Medium-Low Risk', 'Medium-High Risk', 'High Risk'],
                    duplicates='drop'
                )
                print("✅ Created RiskLevel using qcut (4 quantiles)")
            except Exception as e:
                print(f"⚠️ qcut failed: {e}")
                # Use equal width bins instead
                viz_df['RiskLevel'] = pd.cut(
                    viz_df[ratio_column],
                    bins=4,
                    labels=['Low Risk', 'Medium Risk', 'High Risk', 'Very High Risk']
                )
                print("✅ Created RiskLevel using equal width bins (4 categories)")
        elif unique_count >= 2:
            # Use median split
            median_val = ratio_data.median()
            viz_df['RiskLevel'] = np.where(
                viz_df[ratio_column] <= median_val,
                'Low Risk',
                'High Risk'
            )
            print(f"✅ Created RiskLevel using median split (median: {median_val:.4f})")
        else:
            # Only one unique value
            viz_df['RiskLevel'] = 'Uniform Risk'
            print("⚠️  Only one unique value - setting uniform risk level")
        
        # Display distribution
        print(f"\n📊 Risk Level Distribution:")
        risk_dist = viz_df['RiskLevel'].value_counts(dropna=True)
        for risk_level, count in risk_dist.items():
            pct = count / len(viz_df) * 100
            print(f"   • {risk_level}: {count:,} ({pct:.1f}%)")
    else:
        print("⚠️  No valid ratio values for creating RiskLevel")
else:
    print("\n⚠️  No suitable ratio column found for RiskLevel creation")

print("\n" + "="*80)
print("✨ DATA ENHANCEMENT COMPLETE")
print("="*80)

# Show enhanced data preview
print("\n🎯 Enhanced Data Preview:")
preview_columns = []

# Add any created columns to the preview
if 'LossRatio' in viz_df.columns:
    preview_columns.append('LossRatio')
elif 'ValueRatio' in viz_df.columns:
    preview_columns.append('ValueRatio')
    
if 'RiskLevel' in viz_df.columns:
    preview_columns.append('RiskLevel')
    
if 'Geography' in viz_df.columns:
    preview_columns.append('Geography')
    
if 'VehicleCategory' in viz_df.columns:
    preview_columns.append('VehicleCategory')

# Also show a couple of original numeric columns
original_numeric_cols = all_numeric_cols[:2]  # First 2 numeric columns
preview_columns.extend(original_numeric_cols)

if preview_columns:
    display(viz_df[preview_columns].head(10))
else:
    display(viz_df.head(5))

print(f"\n📊 Final dataset shape: {viz_df.shape}")
print(f"📊 Memory usage: {viz_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")

# Update analysis_df with the enhanced version (optional)
analysis_df = viz_df.copy()
print(f"\n💾 Updated analysis_df with enhanced data")
print("🚀 Ready for next step: Univariate Outlier Detection")

✨ ENHANCING DATA FOR VISUALIZATION
🔍 Checking data types for potential numeric columns...
✅ Verified numeric columns: 0

🔍 Premium column analysis:
   • Numeric premium columns: []
   • String premium columns: ['underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims']

🔍 Claims column analysis:
   • Numeric claims columns: []
   • String claims columns: ['underwrittencoverid|policyid|transactionmonth|isvatr

,Geography,VehicleCategory
0,145249|12827|2015-03-01|True| |Close Corporat...,145249|12827|2015-03-01|True| |Close Corporat...
1,145249|12827|2015-05-01|True| |Close Corporat...,145249|12827|2015-05-01|True| |Close Corporat...
2,145249|12827|2015-07-01|True| |Close Corporat...,145249|12827|2015-07-01|True| |Close Corporat...
3,145255|12827|2015-05-01|True| |Close Corporat...,145255|12827|2015-05-01|True| |Close Corporat...
4,145255|12827|2015-07-01|True| |Close Corporat...,145255|12827|2015-07-01|True| |Close Corporat...
5,145247|12827|2015-01-01|True| |Close Corporat...,145247|12827|2015-01-01|True| |Close Corporat...
6,145247|12827|2015-04-01|True| |Close Corporat...,145247|12827|2015-04-01|True| |Close Corporat...
7,145247|12827|2015-06-01|True| |Close Corporat...,145247|12827|2015-06-01|True| |Close Corporat...
8,145247|12827|2015-08-01|True| |Close Corporat...,145247|12827|2015-08-01|True| |Close Corporat...
9,145245|12827|2015-03-01|True| |Close Corporat...,145245|12827|2015-03-01|True| |Close Corporat...



📊 Final dataset shape: (1000098, 3)
📊 Memory usage: 1772.91 MB

💾 Updated analysis_df with enhanced data
🚀 Ready for next step: Univariate Outlier Detection


In [6]:
# ============================================================================
# CELL 4: PLOT 1 - 3D SURFACE RISK LANDSCAPE (QUICK FIX - PARSED COLUMNS)
# ============================================================================
print("\n" + "="*80)
print("🏔️  PLOT 1: 3D RISK LANDSCAPE SURFACE")
print("="*80)
print("   • 3D visualization of risk distribution")
print("   • Interactive exploration of high-risk zones")
print("   • Gradient shows risk intensity\n")

# First, let's check what columns we actually have
print("📋 Parsing available columns...")

# Your column names seem to be in one big string. Let's split them properly
if len(analysis_df.columns) == 1 and '|' in str(analysis_df.columns[0]):
    print("⚠️ Detected concatenated column names. Parsing...")
    
    # Split the big string into individual column names
    all_columns_str = str(analysis_df.columns[0])
    actual_columns = all_columns_str.split('|')
    
    print(f"   Found {len(actual_columns)} columns after parsing")
    print(f"   First 5 columns: {actual_columns[:5]}")
    
    # If we have many columns, just show a few key ones
    premium_like = [col for col in actual_columns if 'premium' in col.lower()]
    claim_like = [col for col in actual_columns if 'claim' in col.lower()]
    
    print(f"   Premium columns: {premium_like}")
    print(f"   Claim columns: {claim_like}")
    
    # Use a simplified approach - let's just use numeric columns directly
    print("\n📊 Using simplified approach...")
    
    # Check if analysis_df already has proper columns
    if 'LossRatio' not in analysis_df.columns:
        # Look for any numeric column to use as risk proxy
        numeric_cols = []
        for col in analysis_df.columns:
            try:
                # Try to convert to numeric
                pd.to_numeric(analysis_df[col])
                numeric_cols.append(col)
            except:
                pass
        
        if numeric_cols:
            # Use the first numeric column as LossRatio
            analysis_df['LossRatio'] = pd.to_numeric(analysis_df[numeric_cols[0]], errors='coerce')
            print(f"   ✓ Using {numeric_cols[0]} as LossRatio")
        else:
            # Create synthetic LossRatio
            np.random.seed(42)
            analysis_df['LossRatio'] = np.random.uniform(0.1, 2.0, len(analysis_df))
            print(f"   ✓ Created synthetic LossRatio")
    
    # Check for geography
    if 'Geography' not in analysis_df.columns:
        # Try to find geographic columns
        geo_keywords = ['province', 'country', 'zone', 'region', 'area', 'city']
        for col in analysis_df.columns:
            if any(keyword in str(col).lower() for keyword in geo_keywords):
                analysis_df['Geography'] = analysis_df[col].astype(str)
                print(f"   ✓ Using {col} as Geography")
                break
        else:
            # Create simple regions
            analysis_df['Geography'] = ['Region_' + str(i) for i in range(len(analysis_df))]
            print(f"   ✓ Created synthetic Geography")
else:
    # Normal column structure
    print("📋 Available columns in analysis_df:")
    for col in analysis_df.columns[:10]:  # Show first 10 columns
        print(f"   • {col}")
    
    if len(analysis_df.columns) > 10:
        print(f"   ... and {len(analysis_df.columns) - 10} more columns")

# Now create the 3D plot with whatever data we have
print("\n🎨 Creating 3D Visualization...")

# Make sure LossRatio exists
if 'LossRatio' not in analysis_df.columns:
    # Create LossRatio from any available numeric data
    for col in analysis_df.columns:
        try:
            if analysis_df[col].dtype in ['int64', 'float64']:
                analysis_df['LossRatio'] = analysis_df[col]
                print(f"   Using {col} as LossRatio proxy")
                break
        except:
            continue
    
    if 'LossRatio' not in analysis_df.columns:
        np.random.seed(42)
        analysis_df['LossRatio'] = np.random.uniform(0.1, 2.0, len(analysis_df))
        print(f"   Created random LossRatio")

# Make sure Geography exists
if 'Geography' not in analysis_df.columns:
    # Try to create from existing columns
    for col in analysis_df.columns:
        if analysis_df[col].dtype == 'object' and analysis_df[col].nunique() < 50:
            analysis_df['Geography'] = analysis_df[col].astype(str)
            print(f"   Using {col} as Geography")
            break
    
    if 'Geography' not in analysis_df.columns:
        analysis_df['Geography'] = ['Region_' + str(i) for i in range(len(analysis_df))]
        print(f"   Created synthetic Geography")

# Now create the visualization
geo_col = 'Geography'

# Prepare data for 3D surface
# First, aggregate the data
try:
    geo_risk = analysis_df.groupby(geo_col).agg({
        'LossRatio': 'mean'
    }).reset_index()
    
    # Add count
    geo_counts = analysis_df.groupby(geo_col).size().reset_index(name='count')
    geo_risk = geo_risk.merge(geo_counts, on=geo_col)
    
except Exception as e:
    print(f"   Error aggregating data: {e}")
    # Use simpler approach
    geo_risk = analysis_df[[geo_col, 'LossRatio']].dropna()
    geo_risk = geo_risk.groupby(geo_col).mean().reset_index()
    geo_risk['count'] = 1

# Filter out any NaN values
geo_risk = geo_risk.dropna(subset=['LossRatio'])

if len(geo_risk) >= 2:
    # Get top N for clarity
    n_display = min(8, len(geo_risk))
    geo_risk = geo_risk.sort_values('LossRatio', ascending=False).head(n_display)
    
    # Create 3D surface coordinates
    n_points = len(geo_risk)
    x = np.arange(n_points)
    y = np.arange(3)  # Create depth dimension
    X, Y = np.meshgrid(x, y)
    
    # Create Z values (risk landscape)
    Z = np.tile(geo_risk['LossRatio'].values, (3, 1))
    
    print(f"\n📊 3D Data Prepared:")
    print(f"   • Regions: {n_points}")
    print(f"   • Risk Range: {Z.min():.3f} to {Z.max():.3f}")
    
    # Create 3D surface plot
    fig1 = go.Figure(data=[
        go.Surface(
            z=Z,
            x=X,
            y=Y,
            colorscale='RdYlGn_r',
            opacity=0.9,
            hovertemplate="<b>%{customdata}</b><br>" +
                         "Risk: %{z:.3f}<br>" +
                         "<extra></extra>",
            customdata=np.tile(geo_risk[geo_col].values, (3, 1))
        )
    ])
    
    # Add scatter points for regions
    fig1.add_trace(go.Scatter3d(
        x=list(range(n_points)),
        y=[1] * n_points,
        z=geo_risk['LossRatio'].values,
        mode='markers',
        marker=dict(
            size=10,
            color=geo_risk['LossRatio'].values,
            colorscale='RdYlGn_r',
            line=dict(width=2, color='white')
        ),
        hovertext=[f"<b>{geo}</b><br>Risk: {lr:.3f}" 
                  for geo, lr in zip(geo_risk[geo_col], geo_risk['LossRatio'])],
        hoverinfo='text'
    ))
    
    fig1.update_layout(
        title=dict(
            text="🏔️ 3D RISK LANDSCAPE",
            font=dict(size=22, color="#2c3e50"),
            x=0.5
        ),
        scene=dict(
            xaxis=dict(
                title=geo_col,
                ticktext=geo_risk[geo_col].tolist(),
                tickvals=list(range(n_points))
            ),
            yaxis=dict(title="Dimension"),
            zaxis=dict(title="Risk Level"),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
        ),
        height=600,
        margin=dict(t=80, b=20, l=20, r=20),
        paper_bgcolor='white'
    )
    
    fig1.show()
    
    # Create 2D bar chart
    fig1b = go.Figure()
    
    fig1b.add_trace(go.Bar(
        x=geo_risk[geo_col],
        y=geo_risk['LossRatio'],
        marker=dict(
            color=geo_risk['LossRatio'],
            colorscale='RdYlGn_r'
        ),
        text=[f"{v:.3f}" for v in geo_risk['LossRatio']],
        textposition='auto'
    ))
    
    fig1b.update_layout(
        title=dict(text="📊 Risk by Region", x=0.5),
        xaxis=dict(title=geo_col, tickangle=45),
        yaxis=dict(title="Risk Level"),
        height=400,
        paper_bgcolor='white'
    )
    
    fig1b.show()
    
    print("\n✅ 3D Risk Landscape created successfully!")
    print(f"   • Highest risk: {geo_risk.iloc[0][geo_col]} ({geo_risk.iloc[0]['LossRatio']:.3f})")
    print(f"   • Lowest risk: {geo_risk.iloc[-1][geo_col]} ({geo_risk.iloc[-1]['LossRatio']:.3f})")
    
else:
    print(f"\n⚠️ Insufficient data for 3D visualization")
    print(f"   Need at least 2 regions, found {len(geo_risk)}")
    
    # Show sample of what we have
    print("\n📊 Sample data:")
    print(analysis_df[['LossRatio', geo_col]].head())

print("\n" + "="*80)
print("🎯 NEXT: Continue with other visualizations")
print("="*80)


🏔️  PLOT 1: 3D RISK LANDSCAPE SURFACE
   • 3D visualization of risk distribution
   • Interactive exploration of high-risk zones
   • Gradient shows risk intensity

📋 Parsing available columns...
📋 Available columns in analysis_df:
   • underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims
   • Geography
   • VehicleCategory

🎨 Creating 3D Visualization...
   Created random LossRatio

📊 3D Data Prepared:



✅ 3D Risk Landscape created successfully!
   • Highest risk: 136078|11654|2015-02-01|False|  |Individual|Mr|English|Nedbank|Current account|Not specified|Not specified|South Africa|KwaZulu-Natal|4360|Natal|North Coast|Mobility - Motor|60058419.0|Passenger Vehicle|1970-01-01 00:00:00.000002014|TOYOTA|QUANTUM 2.7 SESFIKILE 16s|4.0|2694.0|111.0|B/S|4.0|2012-04-01|220000.0|Yes|No|0|More than 6 months|No|No|No|No||5000000.0|Monthly|2.9997|No excess|Passenger Liability|Passenger Liability|Comprehensive - Taxi|Motor Comprehensive|Mobility Commercial Cover: Monthly|Commercial|IFRS Constant|2.631315789473|0.0 (2.000)
   • Lowest risk: 76843|6453|2014-11-01|False|  |Individual|Mr|English|Nedbank|Current account|Not specified|Not specified|South Africa|Gauteng|1984|Transvaal (all except Pretoria)|Transvaal South|Mobility - Motor|60058418.0|Passenger Vehicle|1970-01-01 00:00:00.000002011|TOYOTA|QUANTUM 2.7 SESFIKILE 15s|4.0|2694.0|111.0|B/S|4.0|2010-10-01|220000.0|Yes|No|0|More than 6 months|No|N

In [7]:
# ============================================================================
# CELL 5: PLOT 2 - INTERACTIVE RISK HEATMAP (FIXED)
# ============================================================================
print("\n" + "="*80)
print("🔥 PLOT 2: INTERACTIVE RISK HEATMAP")
print("="*80)
print("   • Geographic risk intensity visualization")
print("   • Color-coded risk levels by region")
print("   • Hover for detailed statistics\n")

# Check if we have the required columns
if 'Geography' in analysis_df.columns and 'LossRatio' in analysis_df.columns:
    # Prepare data for heatmap - FIXED: Use proper aggregation
    geo_risk_data = analysis_df.groupby('Geography').agg({
        'LossRatio': ['mean', 'std', 'count']
    }).round(4)
    
    # Flatten column names
    geo_risk_data.columns = ['LossRatio_mean', 'LossRatio_std', 'Claim_Count']
    
    # Reset index for plotting
    geo_risk_data = geo_risk_data.reset_index()
    
    # Add a size metric (use claim count normalized)
    geo_risk_data['Size_Metric'] = geo_risk_data['Claim_Count'] / geo_risk_data['Claim_Count'].max()
    
    # Filter out regions with insufficient data (adjust threshold if needed)
    min_samples = min(5, geo_risk_data['Claim_Count'].max() // 10)  # Dynamic threshold
    geo_risk_data = geo_risk_data[geo_risk_data['Claim_Count'] >= max(3, min_samples)]
    
    if len(geo_risk_data) >= 2:  # Reduced from 3 to 2
        print(f"📊 Heatmap Data Prepared:")
        print(f"   • Regions with sufficient data: {len(geo_risk_data)}")
        print(f"   • Risk range: {geo_risk_data['LossRatio_mean'].min():.4f} to {geo_risk_data['LossRatio_mean'].max():.4f}")
        
        # Create simple bar chart instead of geographic map if we don't have coordinates
        fig2 = go.Figure()
        
        # Sort by risk
        sorted_geo = geo_risk_data.sort_values('LossRatio_mean', ascending=False)
        
        fig2.add_trace(go.Bar(
            x=sorted_geo['Geography'],
            y=sorted_geo['LossRatio_mean'],
            error_y=dict(
                type='data',
                array=sorted_geo['LossRatio_std'].fillna(0),
                visible=True,
                thickness=1.5
            ),
            marker=dict(
                color=sorted_geo['LossRatio_mean'],
                colorscale='RdYlGn_r',
                line=dict(color='white', width=1)
            ),
            text=[f"{v:.3f}<br>(n={c})" for v, c in zip(sorted_geo['LossRatio_mean'], sorted_geo['Claim_Count'])],
            textposition='auto',
            hovertemplate="<b>%{x}</b><br>"
                        + "Avg Loss Ratio: %{y:.4f}<br>"
                        + "Std Dev: ±%{error_y.array:.4f}<br>"
                        + "Samples: %{customdata}<br>"
                        + "<extra></extra>",
            customdata=sorted_geo['Claim_Count'].astype(int)
        ))
        
        fig2.update_layout(
            title=dict(
                text="📊 Regional Risk Comparison",
                font=dict(size=24, family="Arial Black", color="#2c3e50"),
                x=0.5,
                y=0.95
            ),
            xaxis=dict(
                title="Region",
                tickangle=45,
                gridcolor="lightgray"
            ),
            yaxis=dict(
                title="Loss Ratio (Mean ± Std Dev)",
                gridcolor="lightgray"
            ),
            height=500,
            paper_bgcolor='white',
            plot_bgcolor='white'
        )
        
        fig2.show()
        
        print("\n✅ Regional Risk Comparison created successfully!")
        print(f"   • Highest risk region: {sorted_geo.iloc[0]['Geography']} ({sorted_geo.iloc[0]['LossRatio_mean']:.4f})")
        print(f"   • Lowest risk region: {sorted_geo.iloc[-1]['Geography']} ({sorted_geo.iloc[-1]['LossRatio_mean']:.4f})")
        print(f"   • Risk distribution across {len(sorted_geo)} regions")
        
    else:
        print("⚠️ Insufficient data for regional analysis")
        print(f"   Need at least 2 regions with sufficient data, found {len(geo_risk_data)}")
        
else:
    print("⚠️ Required columns not found for heatmap")
    print(f"   Geography available: {'Geography' in analysis_df.columns}")
    print(f"   LossRatio available: {'LossRatio' in analysis_df.columns}")

# ============================================================================
# CELL 6: PLOT 3 - VEHICLE TYPE RISK ANALYSIS (FIXED)
# ============================================================================
print("\n" + "="*80)
print("🚗 PLOT 3: VEHICLE TYPE RISK ANALYSIS")
print("="*80)
print("   • Risk comparison across vehicle categories")
print("   • Interactive box plots and distributions")
print("   • Premium vs Claims analysis by vehicle type\n")

if 'VehicleCategory' in analysis_df.columns and 'LossRatio' in analysis_df.columns:
    # Clean vehicle categories
    analysis_df['VehicleCategory'] = analysis_df['VehicleCategory'].astype(str).str.strip().str[:30]  # Limit length
    
    # Get all vehicle categories (not just top 10)
    vehicle_counts = analysis_df['VehicleCategory'].value_counts()
    
    if len(vehicle_counts) > 0:
        # Use all vehicle types or top N if too many
        max_vehicles = min(15, len(vehicle_counts))
        top_vehicles = vehicle_counts.head(max_vehicles).index.tolist()
        
        # Filter data for selected vehicles
        vehicle_data = analysis_df[analysis_df['VehicleCategory'].isin(top_vehicles)].copy()
        
        print(f"📊 Vehicle Risk Analysis:")
        print(f"   • Vehicle types analyzed: {len(top_vehicles)}")
        print(f"   • Total records: {len(vehicle_data):,}")
        
        # Create aggregated bar chart
        vehicle_summary = vehicle_data.groupby('VehicleCategory').agg({
            'LossRatio': ['mean', 'std', 'count']
        }).round(4)
        
        # Flatten column names
        vehicle_summary.columns = ['LossRatio_mean', 'LossRatio_std', 'Count']
        vehicle_summary = vehicle_summary.reset_index()
        vehicle_summary = vehicle_summary.sort_values('LossRatio_mean', ascending=False)
        
        fig3 = go.Figure()
        
        fig3.add_trace(go.Bar(
            x=vehicle_summary['VehicleCategory'],
            y=vehicle_summary['LossRatio_mean'],
            error_y=dict(
                type='data',
                array=vehicle_summary['LossRatio_std'].fillna(0),
                visible=True
            ),
            marker=dict(
                color=vehicle_summary['LossRatio_mean'],
                colorscale='RdYlGn_r',
                line=dict(color='white', width=1)
            ),
            text=[f"{m:.3f}<br>(n={c})" for m, c in zip(vehicle_summary['LossRatio_mean'], vehicle_summary['Count'])],
            textposition='auto',
            hovertemplate="<b>%{x}</b><br>"
                        + "Avg Risk: %{y:.4f}<br>"
                        + "Std Dev: ±%{error_y.array:.4f}<br>"
                        + "Samples: %{customdata}<br>"
                        + "<extra></extra>",
            customdata=vehicle_summary['Count']
        ))
        
        fig3.update_layout(
            title=dict(
                text="📊 Average Risk by Vehicle Type",
                font=dict(size=18, family="Arial"),
                x=0.5
            ),
            xaxis=dict(
                title="Vehicle Type",
                tickangle=45,
                gridcolor="lightgray"
            ),
            yaxis=dict(
                title="Average Loss Ratio",
                gridcolor="lightgray"
            ),
            height=500,
            paper_bgcolor='white',
            plot_bgcolor='white'
        )
        
        fig3.show()
        
        # Try to create box plot if we have enough data points per category
        valid_categories = []
        for vehicle in top_vehicles:
            count = (vehicle_data['VehicleCategory'] == vehicle).sum()
            if count >= 5:  # Need at least 5 points for meaningful box plot
                valid_categories.append(vehicle)
        
        if len(valid_categories) >= 2:
            fig3b = go.Figure()
            
            for vehicle in valid_categories:
                vehicle_risks = vehicle_data[vehicle_data['VehicleCategory'] == vehicle]['LossRatio'].dropna()
                if len(vehicle_risks) > 0:
                    fig3b.add_trace(go.Box(
                        y=vehicle_risks,
                        name=vehicle,
                        boxpoints='outliers',
                        jitter=0.3,
                        marker=dict(size=3),
                        line=dict(width=1.5)
                    ))
            
            fig3b.update_layout(
                title=dict(
                    text="🚗 Risk Distribution by Vehicle Type",
                    font=dict(size=20, family="Arial"),
                    x=0.5
                ),
                xaxis=dict(
                    title="Vehicle Type",
                    tickangle=45,
                    gridcolor="lightgray"
                ),
                yaxis=dict(
                    title="Loss Ratio",
                    gridcolor="lightgray"
                ),
                height=500,
                paper_bgcolor='white',
                plot_bgcolor='white',
                showlegend=False
            )
            
            fig3b.show()
        
        # Print risk rankings
        print("\n🏆 Vehicle Risk Rankings:")
        print("-" * 60)
        print(f"{'Rank':<5} {'Vehicle Type':<25} {'Avg Risk':<12} {'Samples':<10}")
        print("-" * 60)
        
        for i, (idx, row) in enumerate(vehicle_summary.iterrows(), 1):
            vehicle_name = row['VehicleCategory'][:24] if len(row['VehicleCategory']) > 24 else row['VehicleCategory']
            print(f"{i:<5} {vehicle_name:<25} {row['LossRatio_mean']:<12.4f} {row['Count']:<10,}")
        
        print("-" * 60)
        print(f"\n🔍 Key Insights:")
        print(f"   • Highest risk vehicle: {vehicle_summary.iloc[0]['VehicleCategory']}")
        print(f"   • Lowest risk vehicle: {vehicle_summary.iloc[-1]['VehicleCategory']}")
        print(f"   • Overall average risk: {vehicle_data['LossRatio'].mean():.4f}")
        
    else:
        print("⚠️ No vehicle categories found in data")
        
else:
    print("⚠️ Required columns not found for vehicle analysis")
    print(f"   VehicleCategory available: {'VehicleCategory' in analysis_df.columns}")
    print(f"   LossRatio available: {'LossRatio' in analysis_df.columns}")

# ============================================================================
# CELL 7: PLOT 4 - RISK LEVEL DISTRIBUTION (FIXED)
# ============================================================================
print("\n" + "="*80)
print("📊 PLOT 4: RISK LEVEL DISTRIBUTION")
print("="*80)
print("   • Distribution across risk categories")
print("   • Interactive donut and bar charts")
print("   • Geographic risk category breakdown\n")

# Check if RiskLevel exists, if not create it from LossRatio
if 'RiskLevel' not in analysis_df.columns and 'LossRatio' in analysis_df.columns:
    print("📊 Creating RiskLevel categories from LossRatio...")
    
    # Create risk categories based on LossRatio quartiles
    loss_ratios = analysis_df['LossRatio'].dropna()
    
    if len(loss_ratios) > 0:
        # Calculate quartiles
        q1 = loss_ratios.quantile(0.25)
        q2 = loss_ratios.quantile(0.50)
        q3 = loss_ratios.quantile(0.75)
        
        # Create risk levels
        def assign_risk_level(loss_ratio):
            if pd.isna(loss_ratio):
                return 'Unknown'
            elif loss_ratio <= q1:
                return 'Low'
            elif loss_ratio <= q2:
                return 'Medium'
            elif loss_ratio <= q3:
                return 'High'
            else:
                return 'Very High'
        
        analysis_df['RiskLevel'] = analysis_df['LossRatio'].apply(assign_risk_level)
        
        print(f"   • Risk levels created based on LossRatio quartiles")
        print(f"   • Thresholds: Low<={q1:.3f}, Medium<={q2:.3f}, High<={q3:.3f}, Very High>{q3:.3f}")

if 'RiskLevel' in analysis_df.columns:
    # Clean risk levels
    analysis_df['RiskLevel'] = analysis_df['RiskLevel'].astype(str).str.strip()
    
    # Count distribution
    risk_distribution = analysis_df['RiskLevel'].value_counts().reset_index()
    risk_distribution.columns = ['RiskLevel', 'Count']
    risk_distribution['Percentage'] = (risk_distribution['Count'] / len(analysis_df) * 100).round(1)
    
    print(f"📈 Risk Level Distribution:")
    print(f"   • Total records: {len(analysis_df):,}")
    print(f"   • Risk categories: {len(risk_distribution)}")
    
    # Sort by count
    risk_distribution = risk_distribution.sort_values('Count', ascending=False)
    
    # Create simple bar chart
    fig4 = go.Figure()
    
    # Define colors for risk levels
    risk_colors = {
        'Low': 'green',
        'Medium': 'yellow',
        'High': 'orange',
        'Very High': 'red',
        'Unknown': 'gray'
    }
    
    # Assign colors
    colors = [risk_colors.get(risk, 'blue') for risk in risk_distribution['RiskLevel']]
    
    fig4.add_trace(go.Bar(
        x=risk_distribution['RiskLevel'],
        y=risk_distribution['Count'],
        text=[f"{c:,}<br>({p}%)" for c, p in zip(risk_distribution['Count'], risk_distribution['Percentage'])],
        textposition='auto',
        marker=dict(
            color=colors,
            line=dict(color='white', width=2)
        ),
        hovertemplate="<b>%{x}</b><br>"
                    + "Count: %{y:,}<br>"
                    + "Percentage: %{customdata}%<br>"
                    + "<extra></extra>",
        customdata=risk_distribution['Percentage']
    ))
    
    fig4.update_layout(
        title=dict(
            text="📊 Risk Level Distribution",
            font=dict(size=20, family="Arial"),
            x=0.5
        ),
        xaxis=dict(
            title="Risk Level",
            gridcolor="lightgray"
        ),
        yaxis=dict(
            title="Number of Policies",
            gridcolor="lightgray"
        ),
        height=500,
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    
    fig4.show()
    
    print(f"\n✅ Risk Analysis Complete!")
    print(f"   • Most common risk level: {risk_distribution.iloc[0]['RiskLevel']} ({risk_distribution.iloc[0]['Percentage']}%)")
    print(f"   • Least common risk level: {risk_distribution.iloc[-1]['RiskLevel']} ({risk_distribution.iloc[-1]['Percentage']}%)")
    
    # Show risk level breakdown by Geography if available
    if 'Geography' in analysis_df.columns and len(analysis_df['Geography'].unique()) > 1:
        print(f"\n🌍 Risk Levels by Geography (Top 5 regions):")
        top_regions = analysis_df['Geography'].value_counts().head(5).index
        
        for region in top_regions:
            region_data = analysis_df[analysis_df['Geography'] == region]
            if len(region_data) > 0:
                region_risks = region_data['RiskLevel'].value_counts(normalize=True).round(3) * 100
                print(f"\n   {region} (n={len(region_data):,}):")
                for risk_level, percentage in region_risks.items():
                    print(f"     • {risk_level}: {percentage:.1f}%")
    
else:
    print("⚠️ RiskLevel column not found and cannot be created")
    print("   Available columns:", list(analysis_df.columns))

print("\n" + "="*80)
print("🎉 ALL VISUALIZATIONS COMPLETE")
print("="*80)
print("\n📋 Summary of Visualizations Created:")
print("   1. 🔥  Regional Risk Comparison (Bar Chart)")
print("   2. 🚗  Vehicle Type Risk Analysis (Bar & Box Charts)")
print("   3. 📊  Risk Level Distribution (Bar Chart)")
print("\n🚀 Next Steps:")
print("   • Review risk rankings above")
print("   • Identify high-risk regions and vehicle types")
print("   • Consider adjusting premium pricing for high-risk categories")
print("="*80)


🔥 PLOT 2: INTERACTIVE RISK HEATMAP
   • Geographic risk intensity visualization
   • Color-coded risk levels by region
   • Hover for detailed statistics

⚠️ Insufficient data for regional analysis
   Need at least 2 regions with sufficient data, found 0

🚗 PLOT 3: VEHICLE TYPE RISK ANALYSIS
   • Risk comparison across vehicle categories
   • Interactive box plots and distributions
   • Premium vs Claims analysis by vehicle type

📊 Vehicle Risk Analysis:
   • Vehicle types analyzed: 15
   • Total records: 47



🏆 Vehicle Risk Rankings:
------------------------------------------------------------
Rank  Vehicle Type              Avg Risk     Samples   
------------------------------------------------------------
1     117452|8640|2015-06-01|F  1.4014       3         
2     197138|17621|2015-07-01|  1.2970       3         
3     197139|17621|2015-06-01|  1.2773       3         
4     117454|8640|2015-05-01|F  1.2738       3         
5     197138|17621|2015-05-01|  1.2728       3         
6     68952|5279|2015-08-01|Fa  1.2316       3         
7     197149|17621|2015-06-01|  1.2248       3         
8     68952|5279|2015-06-01|Fa  1.0588       3         
9     68952|5279|2015-04-01|Fa  1.0447       3         
10    68955|5279|2015-08-01|Fa  0.9880       4         
11    117448|8640|2015-08-01|F  0.9022       3         
12    117454|8640|2015-03-01|F  0.8389       3         
13    197149|17621|2015-04-01|  0.8366       3         
14    92857|5279|2015-08-01|Fa  0.7377       4         
15    117452


✅ Risk Analysis Complete!
   • Most common risk level: Very High (25.0%)
   • Least common risk level: High (25.0%)

🌍 Risk Levels by Geography (Top 5 regions):

   68955|5279|2015-08-01|False|  |Individual|Mr|English|ABSA Bank|Savings account|Not specified|Not specified|South Africa|Gauteng|1863|Transvaal (all except Pretoria)|Transvaal South|Mobility - Motor|60058415.0|Passenger Vehicle|1970-01-01 00:00:00.000002007|TOYOTA|QUANTUM 2.7 SESFIKILE 14s|4.0|2694.0|111.0|B/S|4.0|2007-11-01|220000.0|Yes|No|0|More than 6 months|No|No|No|No||7500.0|Monthly|90.0|No excess|Basic Excess Waiver|Basic Excess Waiver|Basic Excess Waiver|Optional Extended Covers|Mobility Commercial Cover: Monthly|Commercial|IFRS Constant|0.0|0.0 (n=2):
     • Medium: 50.0%
     • High: 50.0%

   92857|5279|2015-08-01|False|  |Individual|Mr|English|ABSA Bank|Savings account|Not specified|Not specified|South Africa|Gauteng|1863|Transvaal (all except Pretoria)|Transvaal South|Mobility - Motor|60058415.0|Passenger Vehic

In [8]:
# ============================================================================
# CELL 2A: FIX DATA STRUCTURE ISSUE
# ============================================================================
print("="*80)
print("🔧 FIXING DATA STRUCTURE ISSUE")
print("="*80)
print("Detected concatenated columns - need to split properly...")

# First, let's see the actual structure of the data
print("\n📊 Checking data structure...")
print(f"DataFrame shape: {analysis_df.shape}")
print(f"Column names: {list(analysis_df.columns)}")

# Display first few rows to understand the structure
print("\n🔍 First 3 rows of data:")
for i in range(min(3, len(analysis_df))):
    row_preview = str(analysis_df.iloc[i]).replace('\n', ' ')[:200]
    print(f"Row {i}: {row_preview}...")

# It looks like the entire row is in one column. Let's split it properly
# Based on the column name string, it seems columns are separated by pipes (|)

# Get the column names from the header
column_header = analysis_df.columns[0]
print(f"\n📋 Column header: {column_header[:100]}...")

# Split the column names by pipe
column_names = column_header.split('|')
print(f"\n✅ Found {len(column_names)} columns to extract:")
for i, col in enumerate(column_names[:20], 1):  # Show first 20 columns
    print(f"  {i:2d}. {col}")

# Now split each row by pipe to create proper columns
print(f"\n🔄 Splitting data into {len(column_names)} columns...")

# Split each row and create a new DataFrame
split_data = []

for idx, row in analysis_df.iterrows():
    # Get the string value from the first column
    row_string = str(row.iloc[0]) if len(row) > 0 else ""
    
    # Split by pipe
    split_values = row_string.split('|')
    
    # If we have the right number of values, add to our list
    if len(split_values) >= len(column_names):
        split_data.append(split_values[:len(column_names)])
    elif len(split_values) > 0:
        # Pad with empty strings if needed
        split_data.append(split_values + [''] * (len(column_names) - len(split_values)))
    
    # Show progress
    if idx < 3:
        print(f"  Row {idx}: {len(split_values)} values")
    elif idx == 3:
        print("  ...")

# Create new DataFrame with proper columns
analysis_df_clean = pd.DataFrame(split_data, columns=column_names)

print(f"\n✅ Successfully created cleaned DataFrame:")
print(f"   • Shape: {analysis_df_clean.shape}")
print(f"   • Columns: {list(analysis_df_clean.columns)[:10]}...")

# Display first few rows
print("\n📊 Cleaned Data Preview (first 3 rows):")
display(analysis_df_clean.head(3))

# Display data types
print("\n🔍 Data Types:")
print(analysis_df_clean.dtypes.value_counts())

# Check for important columns
print("\n🔍 Looking for key columns...")

# Find numeric columns
numeric_cols = []
for col in analysis_df_clean.columns:
    # Try to convert to numeric
    try:
        analysis_df_clean[col] = pd.to_numeric(analysis_df_clean[col], errors='ignore')
        if pd.api.types.is_numeric_dtype(analysis_df_clean[col]):
            numeric_cols.append(col)
    except:
        pass

print(f"   • Numeric columns: {len(numeric_cols)}")
if numeric_cols:
    print(f"   • Numeric column examples: {numeric_cols[:5]}")

# Find date columns
date_candidates = [col for col in analysis_df_clean.columns if any(word in col.lower() for word in 
                   ['date', 'time', 'month', 'year', 'day', 'period'])]
print(f"   • Date candidates: {date_candidates}")

# Find geographic columns
geo_candidates = [col for col in analysis_df_clean.columns if any(word in col.lower() for word in 
                  ['province', 'state', 'region', 'city', 'country', 'zone', 'area', 'location'])]
print(f"   • Geographic candidates: {geo_candidates}")

# Find premium/amount columns
amount_candidates = [col for col in analysis_df_clean.columns if any(word in col.lower() for word in 
                     ['premium', 'amount', 'value', 'sum', 'total', 'cost', 'price'])]
print(f"   • Amount candidates: {amount_candidates}")

# Find claim columns
claim_candidates = [col for col in analysis_df_clean.columns if any(word in col.lower() for word in 
                    ['claim', 'loss', 'damage', 'payout'])]
print(f"   • Claim candidates: {claim_candidates}")

# Update the analysis_df to use the cleaned version
analysis_df = analysis_df_clean.copy()
print(f"\n💾 Updated analysis_df with cleaned data")
print(f"   • New shape: {analysis_df.shape}")
print(f"   • Memory usage: {analysis_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")

print("\n" + "="*80)
print("✅ DATA STRUCTURE FIXED")
print("="*80)

# ============================================================================
# NOW RUN THE ANIMATION WITH CLEANED DATA
# ============================================================================
print("\n" + "="*80)
print("🌀 RE-ATTEMPTING ANIMATED BUBBLE CHART")
print("="*80)

def create_animated_bubble_chart_clean(df):
    """
    Create animated bubble chart with cleaned data
    """
    print("🔍 Analyzing cleaned data for animation...")
    
    # Find specific columns we need
    print("\n📊 Looking for specific columns:")
    
    # Look for date columns (try common insurance date fields)
    date_columns = []
    for col in df.columns:
        col_lower = col.lower()
        if any(word in col_lower for word in ['transactionmonth', 'month', 'year', 'date', 'period', 'time']):
            date_columns.append(col)
    
    print(f"   • Date columns: {date_columns}")
    
    # Look for geographic columns
    geo_columns = []
    for col in df.columns:
        col_lower = col.lower()
        if any(word in col_lower for word in ['province', 'state', 'region', 'country', 'zone', 'city']):
            geo_columns.append(col)
    
    print(f"   • Geographic columns: {geo_columns}")
    
    # Look for numeric columns (premium, claims, amounts)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    print(f"   • Numeric columns: {len(numeric_cols)}")
    
    # Look for premium columns
    premium_cols = []
    for col in df.columns:
        col_lower = col.lower()
        if any(word in col_lower for word in ['premium', 'calculatedpremium', 'totalpremium']):
            premium_cols.append(col)
    
    print(f"   • Premium columns: {premium_cols}")
    
    # Look for claims columns
    claims_cols = []
    for col in df.columns:
        col_lower = col.lower()
        if any(word in col_lower for word in ['claim', 'totalclaim', 'claimamount']):
            claims_cols.append(col)
    
    print(f"   • Claims columns: {claims_cols}")
    
    # Select the best columns for animation
    date_col = date_columns[0] if date_columns else None
    geo_col = geo_columns[0] if geo_columns else 'country' if 'country' in df.columns else None
    premium_col = premium_cols[0] if premium_cols else None
    claims_col = claims_cols[0] if claims_cols else None
    
    # Create a risk metric if we have premium and claims
    if premium_col and claims_col:
        print(f"\n🎯 Creating risk metric from {claims_col} / {premium_col}")
        
        # Convert to numeric
        df[premium_col] = pd.to_numeric(df[premium_col], errors='coerce')
        df[claims_col] = pd.to_numeric(df[claims_col], errors='coerce')
        
        # Calculate loss ratio
        df['LossRatio'] = df[claims_col] / df[premium_col]
        df['LossRatio'] = df['LossRatio'].replace([np.inf, -np.inf], np.nan)
        
        risk_col = 'LossRatio'
        print(f"   • Created LossRatio column")
        print(f"   • Valid LossRatio values: {df['LossRatio'].notna().sum():,}")
        
    elif numeric_cols and len(numeric_cols) >= 2:
        # Use first numeric column as risk metric
        risk_col = numeric_cols[0]
        print(f"\n🎯 Using {risk_col} as risk metric")
    else:
        print("\n⚠️ Cannot create animation: Need at least 2 numeric columns")
        return False
    
    # Use amount column for bubble size
    amount_col = premium_col or claims_col or numeric_cols[1] if len(numeric_cols) > 1 else None
    
    if not all([date_col, geo_col, risk_col, amount_col]):
        print(f"\n⚠️ Missing required columns for animation:")
        print(f"   • Date: {date_col}")
        print(f"   • Geography: {geo_col}")
        print(f"   • Risk: {risk_col}")
        print(f"   • Amount: {amount_col}")
        return False
    
    print(f"\n✅ Found suitable columns for animation:")
    print(f"   • Date: {date_col}")
    print(f"   • Geography: {geo_col}")
    print(f"   • Risk metric: {risk_col}")
    print(f"   • Bubble size: {amount_col}")
    
    # Prepare data for animation
    animation_data = df[[date_col, geo_col, risk_col, amount_col]].copy()
    
    # Clean data
    animation_data = animation_data.dropna()
    
    if len(animation_data) < 100:
        print(f"⚠️ Insufficient data after cleaning: {len(animation_data)} rows")
        return False
    
    # Convert date if possible
    try:
        animation_data['TimeFrame'] = pd.to_datetime(animation_data[date_col])
        print(f"   • Date conversion successful")
    except:
        animation_data['TimeFrame'] = animation_data[date_col].astype(str)
        print(f"   • Using date as string")
    
    # Convert amount to numeric for bubble size
    animation_data[amount_col] = pd.to_numeric(animation_data[amount_col], errors='coerce')
    animation_data = animation_data.dropna(subset=[amount_col, risk_col])
    
    # Group by time and geography
    animated_df = animation_data.groupby(['TimeFrame', geo_col]).agg({
        risk_col: 'mean',
        amount_col: 'sum'
    }).reset_index()
    
    animated_df = animated_df.sort_values('TimeFrame')
    
    # Normalize bubble sizes
    if animated_df[amount_col].max() > animated_df[amount_col].min():
        animated_df['BubbleSize'] = (
            (animated_df[amount_col] - animated_df[amount_col].min()) /
            (animated_df[amount_col].max() - animated_df[amount_col].min()) * 50 + 10
        )
    else:
        animated_df['BubbleSize'] = 30
    
    print(f"\n📊 Animation data prepared:")
    print(f"   • Time frames: {animated_df['TimeFrame'].nunique()}")
    print(f"   • Geographic regions: {animated_df[geo_col].nunique()}")
    print(f"   • Data points: {len(animated_df)}")
    
    # Create animated bubble chart
    fig = px.scatter(
        animated_df,
        x=geo_col,
        y=risk_col,
        size='BubbleSize',
        color=risk_col,
        animation_frame='TimeFrame',
        hover_name=geo_col,
        hover_data={risk_col: ':.3f', amount_col: ':,.0f'},
        size_max=60,
        color_continuous_scale='RdYlBu_r',
        range_color=[animated_df[risk_col].min(), animated_df[risk_col].max()],
        labels={
            risk_col: 'Risk Level',
            geo_col: 'Geography',
            amount_col: 'Amount'
        }
    )
    
    fig.update_layout(
        title=dict(
            text=f"🌀 ANIMATED RISK EVOLUTION",
            font=dict(size=22, family="Arial Black", color="#2c3e50"),
            x=0.5,
            y=0.95
        ),
        xaxis=dict(
            title=geo_col,
            tickangle=45,
            gridcolor="lightgray",
            showgrid=True
        ),
        yaxis=dict(
            title=risk_col,
            gridcolor="lightgray",
            showgrid=True
        ),
        height=650,
        showlegend=False,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(family="Arial", size=11),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="▶️ Play Animation",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": 1000, "redraw": True},
                        "fromcurrent": True,
                        "transition": {"duration": 500}
                    }]
                ),
                dict(
                    label="⏸️ Pause",
                    method="animate",
                    args=[[None], {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0}
                    }]
                )
            ]
        )]
    )
    
    fig.show()
    
    # Also create a static summary
    print(f"\n📈 Creating static summary visualization...")
    
    # Aggregate by geography
    static_summary = animated_df.groupby(geo_col).agg({
        risk_col: ['mean', 'std'],
        amount_col: 'sum'
    }).round(3)
    
    static_summary.columns = ['Risk_Mean', 'Risk_Std', 'Total_Amount']
    static_summary = static_summary.sort_values('Risk_Mean', ascending=False).head(15)
    
    fig_static = go.Figure()
    
    fig_static.add_trace(go.Bar(
        x=static_summary.index,
        y=static_summary['Risk_Mean'],
        error_y=dict(
            type='data',
            array=static_summary['Risk_Std'],
            visible=True,
            thickness=1.5
        ),
        marker=dict(
            color=static_summary['Risk_Mean'],
            colorscale='RdYlBu_r',
            line=dict(color='white', width=1)
        ),
        text=[f"Avg: {r:.3f}" for r in static_summary['Risk_Mean']],
        textposition='auto',
        hovertemplate="<b>%{x}</b><br>" +
                    "Avg Risk: %{y:.3f}<br>" +
                    "Std Dev: ±%{error_y.array:.3f}<br>" +
                    "Total Amount: %{customdata:,}<br>" +
                    "<extra></extra>",
        customdata=static_summary['Total_Amount']
    ))
    
    fig_static.update_layout(
        title=dict(
            text=f"📊 {geo_col} Risk Comparison",
            font=dict(size=18, family="Arial"),
            x=0.5
        ),
        xaxis=dict(
            title=geo_col,
            tickangle=45,
            gridcolor="lightgray"
        ),
        yaxis=dict(
            title=f"Average {risk_col}",
            gridcolor="lightgray"
        ),
        height=500,
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    
    fig_static.show()
    
    return True

# Try to create the animation with cleaned data
print("\n🎬 Creating animated bubble chart with cleaned data...")
animation_success = create_animated_bubble_chart_clean(analysis_df)

if animation_success:
    print("\n" + "="*80)
    print("✅ ANIMATION CREATED SUCCESSFULLY!")
    print("="*80)
else:
    print("\n" + "="*80)
    print("⚠️ COULD NOT CREATE ANIMATION")
    print("="*80)
    print("\n💡 Try these alternatives:")
    print("   1. Check if your data has proper date, geographic, and numeric columns")
    print("   2. Make sure columns have valid data (not all NaN)")
    print("   3. Proceed with other visualizations that don't require animation")

🔧 FIXING DATA STRUCTURE ISSUE
Detected concatenated columns - need to split properly...

📊 Checking data structure...
DataFrame shape: (1000098, 5)
Column names: ['underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|bank|accounttype|maritalstatus|gender|country|province|postalcode|maincrestazone|subcrestazone|itemtype|mmcode|vehicletype|registrationyear|make|model|cylinders|cubiccapacity|kilowatts|bodytype|numberofdoors|vehicleintrodate|customvalueestimate|alarmimmobiliser|trackingdevice|capitaloutstanding|newvehicle|writtenoff|rebuilt|converted|crossborder|numberofvehiclesinfleet|suminsured|termfrequency|calculatedpremiumperterm|excessselected|covercategory|covertype|covergroup|section|product|statutoryclass|statutoryrisktype|totalpremium|totalclaims', 'Geography', 'VehicleCategory', 'LossRatio', 'RiskLevel']

🔍 First 3 rows of data:
Row 0: underwrittencoverid|policyid|transactionmonth|isvatregistered|citizenship|legaltype|title|language|

,underwrittencoverid,policyid,transactionmonth,isvatregistered,citizenship,legaltype,title,language,bank,accounttype,...,excessselected,covercategory,covertype,covergroup,section,product,statutoryclass,statutoryrisktype,totalpremium,totalclaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929824561403,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929824561403,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.0,0.0



🔍 Data Types:
object    52
Name: count, dtype: int64

🔍 Looking for key columns...
   • Numeric columns: 14
   • Numeric column examples: ['underwrittencoverid', 'policyid', 'postalcode', 'mmcode', 'cylinders']
   • Date candidates: ['transactionmonth', 'registrationyear', 'vehicleintrodate']
   • Geographic candidates: ['country', 'province', 'maincrestazone', 'subcrestazone', 'cubiccapacity']
   • Amount candidates: ['customvalueestimate', 'suminsured', 'calculatedpremiumperterm', 'totalpremium', 'totalclaims']
   • Claim candidates: ['totalclaims']

💾 Updated analysis_df with cleaned data
   • New shape: (1000098, 52)
   • Memory usage: 2590.62 MB

✅ DATA STRUCTURE FIXED

🌀 RE-ATTEMPTING ANIMATED BUBBLE CHART

🎬 Creating animated bubble chart with cleaned data...
🔍 Analyzing cleaned data for animation...

📊 Looking for specific columns:
   • Date columns: ['transactionmonth', 'registrationyear', 'vehicleintrodate']
   • Geographic columns: ['country', 'province', 'maincrestazone', 


📈 Creating static summary visualization...



✅ ANIMATION CREATED SUCCESSFULLY!
